<a href="https://colab.research.google.com/github/DarshanChikkahanasoge/Voice-classification-/blob/main/voice_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
from matplotlib.pyplot import specgram
from sklearn.metrics import confusion_matrix
import IPython.display as ipd 
import os 
import sys
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from keras.regularizers import l2
import seaborn as sns
from keras.callbacks import EarlyStopping, ModelCheckpoint


from sklearn.metrics import classification_report

In [2]:
import librosa
import soundfile
import glob,pickle
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
emotions={
    'W':'anger',
    'L':'boredom',
    'A':'anxiety',
    'F':'happiness',
    'T':'sadness',
    'E':'disgust',
    'N':'neutral'
}

In [5]:
def extract_feature1(file_name):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
  """
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        stft = np.abs(librosa.stft(X))
        result = np.array([])

        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
        
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
      
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

In [6]:
def load_data1():
  x,y=[],[]
  global max1
  max1=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Berlin/Berlin/wav/*.wav"):
    filename=os.path.basename(file)
    emotion=emotions[filename.split(".")[0][-2]]
    feature=extract_feature1(file)
    if len(feature)>max1:
      max1=len(feature)
    x.append(feature)
    y.append(emotion)
  return x,y

In [7]:
x1,Y1=load_data1()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=370
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=422
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=442
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=396
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=360
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=391
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [8]:
x_train1,x_test1,y_train1,y_test1=train_test_split(np.array(x1),Y1,test_size=0.25,random_state=9)

KNN classifier

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
classifier3=KNeighborsClassifier(n_neighbors=15,metric='minkowski',p=5)

In [12]:
classifier3.fit(x_train1,y_train1)

KNeighborsClassifier(n_neighbors=15, p=5)

In [13]:
y_pred1=classifier3.predict(x_test1)

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test1, y_pred=y_pred1)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 45.52%


MLP Classifier

In [16]:
model_params2 = {
    'alpha': 0.01,
    'batch_size': 35,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}

In [17]:
from sklearn.neural_network import MLPClassifier
model2 = MLPClassifier(**model_params2)

In [18]:
model2.fit(x_train1, y_train1)

MLPClassifier(alpha=0.01, batch_size=35, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [19]:
# predict 25% of data to measure how good we are
y_pred1 = model2.predict(x_test1)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test1, y_pred=y_pred1)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.40%


In [24]:
def anger_data():
  x,y=[],[]
  global maxa
  maxa=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Anger/*.wav"):
    filename=os.path.basename(file)
    emotion='anger'
    feature=extract_feature1(file)
    if len(feature)>maxa:
      maxa=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxa:
      x[i]+=[1]*(maxa-len(x[i]))
  return x,y

In [25]:
x_angry,y_angry=anger_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=470
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=375
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=423
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=404
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=467
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=452
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [26]:
print(len(x_angry))

129


In [27]:
pred_angry=model2.predict(np.array(x_angry))

In [28]:
# calculate the accuracy
accuracy = accuracy_score(y_true=y_angry, y_pred=pred_angry)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 94.57%


In [29]:
def boredom_data():
  x,y=[],[]
  global maxb
  maxb=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Boredom/*.wav"):
    filename=os.path.basename(file)
    emotion='boredom'
    feature=extract_feature1(file)
    if len(feature)>maxb:
      maxb=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxb:
      x[i]+=[1]*(maxb-len(x[i]))
  return x,y

In [30]:
x_boredom,y_boredom=boredom_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=483
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=505
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=434
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=381
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=382
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=399
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [31]:
print(len(x_boredom))

82


In [32]:
pred_boredom=model2.predict(np.array(x_boredom))

In [33]:
# calculate the accuracy
accuracy = accuracy_score(y_true=y_boredom, y_pred=pred_boredom)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 89.02%


In [34]:
def disgust_data():
  x,y=[],[]
  global maxd
  maxd=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Disgust/*.wav"):
    filename=os.path.basename(file)
    emotion='disgust'
    feature=extract_feature1(file)
    if len(feature)>maxd:
      maxd=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxd:
      x[i]+=[1]*(maxd-len(x[i]))
  return x,y

In [35]:
x_disgust,y_disgust=disgust_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=381
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=478
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=457
  n_fft, y.shape[-1]


In [36]:
pred_disgust=model2.predict(np.array(x_disgust))

In [37]:
accuracy = accuracy_score(y_true=y_disgust, y_pred=pred_disgust)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 86.36%


In [38]:
def fear_data():
  x,y=[],[]
  global maxf
  maxf=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Fear/*.wav"):
    filename=os.path.basename(file)
    emotion='anxiety'
    feature=extract_feature1(file)
    if len(feature)>maxf:
      maxf=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxf:
      x[i]+=[1]*(maxf-len(x[i]))
  return x,y

In [39]:
x_fear,y_fear=fear_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=377
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=464
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=396
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=422
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=420
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=415
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [40]:
pred_fear=model2.predict(np.array(x_fear))

In [41]:
accuracy = accuracy_score(y_true=y_fear, y_pred=pred_fear)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 94.03%


In [42]:
def happiness_data():
  x,y=[],[]
  global maxh
  maxh=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Happiness/*.wav"):
    filename=os.path.basename(file)
    emotion='happiness'
    feature=extract_feature1(file)
    if len(feature)>maxh:
      maxh=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxh:
      x[i]+=[1]*(maxh-len(x[i]))
  return x,y

In [43]:
x_happiness,y_happiness=happiness_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=475
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=502
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=425
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=405
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=451
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=491
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [44]:
pred_happiness=model2.predict(np.array(x_happiness))

In [45]:
accuracy = accuracy_score(y_true=y_happiness, y_pred=pred_happiness)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 88.89%


In [46]:
def neutral_data():
  x,y=[],[]
  global maxn
  maxn=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Neutral/*.wav"):
    filename=os.path.basename(file)
    emotion='neutral'
    feature=extract_feature1(file)
    if len(feature)>maxn:
      maxn=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxn:
      x[i]+=[1]*(maxn-len(x[i]))
  return x,y

In [47]:
x_neutral,y_neutral=neutral_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=403
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=360
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=391
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=448
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=442
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=509
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectr

In [48]:
pred_neutral=model2.predict(np.array(x_neutral))

In [49]:
accuracy = accuracy_score(y_true=y_neutral, y_pred=pred_neutral)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 93.67%


In [50]:
def sadness_data():
  x,y=[],[]
  global maxs
  maxs=0
  for file in glob.glob("/gdrive/My Drive/Berlin-2/Sadness/*.wav"):
    filename=os.path.basename(file)
    emotion='sadness'
    feature=extract_feature1(file)
    if len(feature)>maxs:
      maxs=len(feature)
    x.append(feature)
    y.append(emotion)
  for i in range(len(x)):
    if len(x[i])<maxs:
      x[i]+=[1]*(maxs-len(x[i]))
  return x,y

In [51]:
x_sad,y_sad=sadness_data()

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=434
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=499
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=489
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=466
  n_fft, y.shape[-1]


In [52]:
pred_sad=model2.predict(np.array(x_sad))

In [53]:
accuracy = accuracy_score(y_true=y_sad, y_pred=pred_sad)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 91.94%
